In [1]:
import os, cv2
import tensorflow as tf
import matplotlib.pyplot as plt, numpy as np, pandas as pd
from tensorflow import keras

from centernet import CenterNet, decode

from data import VOCDataset
from losses import centernet_loss

In [2]:
input_shape = (512, 512)
backbone = 'resnet50'

epochs = 1
batch_size = 2
buffer_size = batch_size * 5
lr = 1e-2
freeze = True
finetune = True or freeze

data_path = "..\\..\\Centernet\\VOC2007"
train_file = "..\\..\\Centernet\\VOC2007\\ImageSets\\ceph\\train.txt"
val_file = "..\\..\\Centernet\\VOC2007\\ImageSets\\ceph\\val.txt"
test_file = "..\\..\\Centernet\\VOC2007\\ImageSets\\ceph\\test.txt"


train_dataset_raw = VOCDataset(data_path, input_shape, train_file,
                             batch_size, False)
train_dataset = train_dataset_raw.load_dataset()


val_dataset_raw = VOCDataset(data_path, input_shape, val_file,
                             batch_size, False)
val_dataset = val_dataset_raw.load_dataset()

########
test_dataset_raw = VOCDataset(data_path, input_shape, test_file,
                             batch_size, False)
test_dataset = test_dataset_raw.load_dataset()
########

vis_dataset_raw = VOCDataset(data_path, input_shape, train_file, 1, False)
vis_dataset = vis_dataset_raw.load_dataset().repeat()

steps_per_epoch = len(train_dataset_raw) // batch_size

loading the context of data_file ...: 550it [00:00, 551222.75it/s]
loading the context of data_file ...: 50it [00:00, ?it/s]
loading the context of data_file ...: 50it [00:00, ?it/s]
loading the context of data_file ...: 550it [00:00, 550827.89it/s]


In [3]:
ckpt_path = './logs/test/20220621-062329/ckpt.h5'

model = CenterNet(train_dataset_raw.class_names,
                      backbone_weights='imagenet',
                      freeze=freeze,
                      finetune=finetune)

In [4]:
model.build(input_shape=(1, 512, 512, 3))
model.load_weights(ckpt_path)

In [5]:
hm_pred, wh_pred, reg_pred = model.predict(val_dataset)

In [6]:
detections_pred = decode(hm_pred, wh_pred, reg_pred, 1000)

In [7]:
detections_pred.shape

TensorShape([50, 1000, 6])

In [8]:
def point_mse(arr, gt):
    pnt_x = int((arr[0] + arr[2])/2)
    pnt_y = int((arr[1] + arr[3])/2)

    xmin,ymin,xmax,ymax = gt

    gt_x = int((xmin + xmax)/2)
    gt_y = int((ymin + ymax)/2)

    return np.sqrt((gt_y-pnt_y)**2 + (gt_x-pnt_x)**2)

In [15]:
t = detections_pred

mse_result = []

# ['Glabella', 'R3', 'Cervical point', 'Go-2 Ramus down', 'Naison', 'Pogonion']

for i in range(50):
    #topk_x1, topk_y1, topk_x2, topk_y2, scores, class_ids
    Glabella = []
    point_R3 = []
    Nasion = []


    for topk_x1, topk_y1, topk_x2, topk_y2, scores, class_ids in t[i]:

        if(class_ids==0):
            Glabella.append([topk_x1, topk_y1, topk_x2, topk_y2])

        elif(class_ids==1):
            point_R3.append([topk_x1, topk_y1, topk_x2, topk_y2])

        elif(class_ids==3):
            Nasion.append([topk_x1, topk_y1, topk_x2, topk_y2])

    
    #print(len(Glabella))
    
    Glabella = np.array(Glabella)
    point_R3 = np.array(point_R3)
    Nasion = np.array(Nasion)

    # ground truth
    Glabella_gt = val_dataset_raw[i][1][0]
    point_R3_gt = val_dataset_raw[i][1][1]
    Nasion_gt = val_dataset_raw[i][1][3]


    Glabella_mse = point_mse(Glabella[0], Glabella_gt)
    point_R3_mse = point_mse(point_R3[0], point_R3_gt)
    Nasion_mse = point_mse(Nasion[0], Nasion_gt)

    #print(Glabella_mse, point_R3_mse, Nasion_mse)


    mse_result.append([i+1, np.round(Glabella_mse, 4), np.round(point_R3_mse, 4), np.round(Nasion_mse, 4)])


    

In [16]:
import datetime

d = datetime.datetime.now()
today = f'{d.month}_{d.day}'

print(today)

# 이후 strftime() method 사용해볼 것!

6_22


In [17]:
# mse_result_df = pd.DataFrame(mse_result, columns = ['Data_num', 'Glabella', 'R3', 'Nasion'], )
mse_result_df = pd.DataFrame(mse_result, columns = ['Data_num', 'Glabella', 'R3', 'Nasion'], )

mse_result_df.to_csv(f'./csv/mse_{today}.csv', index=False)

In [18]:
mse_result_df

,Data_num,Glabella,R3,Nasion
0,1,5.0000,4.0000,9.0554
1,2,2.2361,3.6056,5.0990
2,3,5.0990,10.4403,7.0711
3,4,4.1231,6.4031,5.0000
4,5,5.8310,4.0000,5.0000
5,6,4.0000,5.0000,12.7279
6,7,1.0000,2.2361,12.1655
7,8,0.0000,6.7082,7.0711
8,9,3.6056,4.4721,5.3852
9,10,3.6056,4.4721,12.5300
